# Humid air calculation using CoolProp

In [1]:
from CoolProp.HumidAirProp import HAPropsSI
import iapws as steam # This is the library to call in for steam calculation
import numpy as np

Calculate the mass of water contained and the enthalpy in the air in an empty room of 1m3. Pressure is 1 atm, and temperature is 25$^\text{o}$C. The air has relative humidity of 50%. Calculate the mass of water vapor at 100% humidity.


In [2]:
p = 101325;
T = 25;
Humid = 0.5;
V = 1;
R_dryair = 286.9;
R_water = 461.4;

Tabs = T + 273.15

hha = HAPropsSI('Hha', 'T', Tabs, 'P', p, 'R', Humid); # Specific enthalpy
W = HAPropsSI('W', 'T', Tabs, 'P', p, 'R', Humid);     # specific humidity
Vha = HAPropsSI('Vha', 'T', Tabs, 'P', p, 'R', Humid); # specific volumer per humid air
Z = HAPropsSI('Z', 'T', Tabs, 'P', p, 'R', Humid)      # compressibility factor

m = V / Vha                                             
R = (R_dryair + R_water * W) / (1 + W)
m1 = p * V / (R * Tabs)                   # We assume ideal gas
m_v = m * W / (1 + W)
H = hha * m

print('The specific enthalpy of humid air: {:3.4e}J/kg Dry Air'.format(hha))
print('The specific humidity: {:3.4e}kg Water/kg Dry Air'.format(W))
print('The enthalpy of the humud air in the room: {:3.4f}J'.format(H))
print('The mass of the air in the room (ideal gas): {:3.4e}kg'.format(m))
print('The mass of the air in the room (real gas): {:3.4e}kg'.format(m1))
print('The mass of the water vapor: {:3.4e}kg'.format(m_v))
print('Compressibility factor: {:3.4f}'.format(Z))

The specific enthalpy of humid air: 4.9928e+04J/kg Dry Air
The specific humidity: 9.9257e-03kg Water/kg Dry Air
The enthalpy of the humud air in the room: 58783.0980J
The mass of the air in the room (ideal gas): 1.1774e+00kg
The mass of the air in the room (real gas): 1.1775e+00kg
The mass of the water vapor: 1.1571e-02kg
Compressibility factor: 0.9996


Now we inject the saturated steam at 8 bar until we reach 80% humidity. Calculate the amount of steam to be injected assuming
  * The room temperature is isothermal.
  * The room is adiabatic

For the isothermal case, the pressure increase is proportional to the mass of the steam. Assuming that partial vapor pressure at the saturated condition is not changed,

In [3]:
err = 1
m_a = m - m_v
m_v0 = m_v
m_s = 0.5 * m_v
while err > 0.0001:
    m_s *= (1 + err)
    m_v = m_v0 + m_s
    m = m_a + m_v
    W = m_v / m_a
    R = (R_dryair + R_water * W) / (1 + W)
    err_Humid_new = 1
    p_new = m * R * Tabs / V
    Humid_new = HAPropsSI('R', 'P', p_new, 'T', Tabs, 'W', W)
    Z = HAPropsSI('Z', 'P', p_new, 'T', Tabs, 'W', W)
    print('Compressibility factor: {}'.format(Z))
    err = abs(1 - Humid_new)
    print('error {}'.format(err))
    
print('Mass of injected steam is ection is {:3.4e}kg'.format(m_s))

Compressibility factor: 0.999561179848814
error 0.0001654472196314094
Compressibility factor: 0.9995611664688188
error 8.274434846400336e-05
Mass of injected steam is ection is 1.1573e-02kg


Now check if we satisfy the saturated condition.

In [4]:
Humid_new = HAPropsSI('R', 'P', p_new, 'T', Tabs, 'W', W)
print('Humidity after steam injection is {:3.2f}%'.format(Humid_new*100))

Humidity after steam injection is 99.99%


For adiabatic case, we need to consider the energy balance and calculate the new temperature as well. Tje dew temperature will increase as the overall temperature increases. The amount of the steam to achieve 100% relative humidity should therefore be calculated iteratively.

In [5]:
# First get the saturated steam object @ 8 bar
sat_steam = steam.IAPWS97(P = 0.8, x = 1)

# Start with injecting the same amount of steam caculated in the isothermal case
err = 0.5
m_s = m_v0*0.5
H_ha = H
n = 1
while err > 0.0001 or err_p_new < 0:
    print('Iteration number: {:d}'.format(n))
    n += 1
    m_s *= 1 + err*2
    print('injected steam {}kg'.format(m_s))
    m_v = m_s + m_v0
    m = m_a + m_v
    W = m_v / m_a
    cp_temp = HAPropsSI('cp', 'P', p, 'T', Tabs, 'W', W)
    R = (R_dryair + R_water * W) / (1 + W)
    H_steam = m_s * sat_steam.h * 1000
    h_new = (H_ha + H_steam) / m
    deltaT = H_steam * (sat_steam.T - Tabs) / sat_steam.T / cp_temp
    T_new = Tabs + deltaT
    p_new = m * R * T_new / V
    print('Initial temperature {}K'.format(T_new))
    print('Initial pressure {}Pa'.format(p_new))
    err_p_new = 1
    while err_p_new > 0.0001:
        T_new = HAPropsSI('T', 'P', p_new, 'Hha', h_new, 'W', W)
        p_new_temp = m * R * T_new / V
        err_p_new = abs(p_new - p_new_temp) / p_new
        p_new = p_new_temp
        print('Updated temperature {}K'.format(T_new))
        print('Updated pressure {}Pa'.format(p_new))
    Humid_new = HAPropsSI('R', 'P', p_new, 'T', T_new, 'W', W)
    print('Humidity {}R%'.format(Humid_new*100))
    err = (1 - Humid_new)

Iteration number: 1
injected steam 0.011571316548412607kg
Initial temperature 308.40841411384554K
Initial pressure 106444.9565626865Pa
Updated temperature 300.28532597767065K
Updated pressure 103641.33083705753Pa
Updated temperature 300.27809456961484K
Updated pressure 103638.83496832904Pa
Humidity 88.78323478794792R%
Iteration number: 2
injected steam 0.014167171366770148kg
Initial temperature 310.68550782233325K
Initial pressure 107602.99570647221Pa
Updated temperature 300.75572803452474K
Updated pressure 104163.91012001227Pa
Updated temperature 300.74667028068376K
Updated pressure 104160.77305238883Pa
Humidity 96.21676511011272R%
Iteration number: 3
injected steam 0.015239126106885688kg
Initial temperature 311.6232498192022K
Initial pressure 108081.90263721105Pa
Updated temperature 300.949075782473K
Updated pressure 104379.72367707327Pa
Updated temperature 300.93923735125514K
Updated pressure 104376.31136311528Pa
Humidity 99.16537312999289R%
Iteration number: 4
injected steam 0.0154

How much heat $\dot{Q}$ is needed to warm up a humid air flow, i.e. the mixture, of $\dot{m}$= 0.5 kg/s to 30$^\text{o}$C, having heater inlet conditions: temperature $T_1 = 20^\text{o}\text{C}$ and humid ratio (moisture content) $x_1 = 0.0088$. Also state: what will the relative humidity $\varphi$ then become? (Exam 2017)

$\dot{Q}$ can be calculated from the heat balance
$$\dot{Q} = \dot{m} \left(h_2 - h_1\right)$$

In [6]:
P = 101325
T1 = 25 + 273.15
T2 = 30 + 273.15
W = 0.0088
mdot = 0.5

# Specific enthalpy
h1 = HAPropsSI('Hha', 'P', P, 'T', T1, 'W', W)   
h2 = HAPropsSI('Hha', 'P', P, 'T', T2, 'W', W)   

Qdot = mdot * (h2 - h1)
print('The required heat input is {:3.2f} W'.format(Qdot))

The required heat input is 2535.07 W


For a humidifier, an apparatus used to increase the humidity of air, we find the inlet air quality to be 30 °C and , having a humid air flow of 0.5 kg/s and x = 0.0088. The humidifier adds 3 g/s water with a temperature of 20 °C to this stream. 
  - What is the temperature of the air out of the humidifier and what is the relative humidity?
  - How much water, if any, is condensed per second and what is the temperature of this water?

We assume that the water added is in liquid form. First we can calculate the maximum water that the air can hold at the water temperature. We assume that the air will be cooled down to the water temperature due to vaporization.

In [7]:
P = 101325
T1 = 30 + 273.15
T2 = 20 + 273.14
dm_1 = 0.5
dm_w = 0.003
W0 = 0.0088

W1 = (dm_1 * W0 / (1 + W0) + dm_w) / (dm_1 / (1 + W0))
W_max = HAPropsSI('W', 'P', P, 'T', T2, 'R', 1)
if W1 > W_max:
    print('The air may be saturated before all the added water become vaporized')
else:
    print('The water may be vaporized depending on the temperature difference')

The air may be saturated before all the added water become vaporized


Therefore, use the energy balance to calculate the maximum heat transfered to water for vaporization and to calculate the maximum mass flow of water to be vaporized. 
$$\dot{m}_\text{water} = \frac { \dot{m}_\text{ha} \left(h_1(T_1) - h_2(T_2)\right)}{h_{fg}}$$

In [8]:
h_fg = 2453.6e3 #J/kg
h1 = HAPropsSI('Hha', 'P', P, 'T', T1, 'R', W0)
h2 = HAPropsSI('Hha', 'P', P, 'T', T2, 'R', W0)
dm_w_vap = dm_1 * (h1 - h2) / h_fg
print('{} kg/s of water will be vaporized'.format(dm_w_vap))

0.00210626495038435 kg/s of water will be vaporized


In [9]:
# Calculation of relative humidity
W1 = (dm_1 * W0 / (1 + W0) + dm_w_vap) / (dm_1 / (1 + W0))
humid = HAPropsSI('R', 'P', P, 'T', T2, 'W', W)
print('The relative humidity of the air will be {:3.2f}%R'.format(humid * 100))

The relative humidity of the air will be 60.22%R
